## Encontre a bandeira

Você consegue adivinhar de qual continente esta bandeira vem?

In [ ]:
from IPython.display import Image
Image('bandeira exercício.png')

Quais variáveis indicariam? Talvez algumas das cores sejam bons indicadores. A presença ou ausência de certas formas pode dar uma dica. Neste projeto, usaremos árvores de decisão para tentar prever o continente de bandeiras com base em vários desses recursos.

O conjunto de dados original está disponível no repositório de aprendizado de máquina UCI:

https://archive.ics.uci.edu/ml/datasets/Flags

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [2]:
#nos ultimos testes comecou a dar timeout na comunicacao com o servidor de onde sao pegos os dados

cols = ['nome','continente','zone', 'area', 'população', 'idioma','religião','barras','listras','cores',
'vermelho','verde','azul','dourado','branco','preto','laranja','cor_predominante','círculos',
'cruzamentos','cruzes','quarters','sol_estrelas','crescent','triangle','icon','animate','text','topleft','botright']

df= pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/flags/flag.data", names = cols)
df.info()

URLError: ignored

O conjunto de dados foi carregado e salvo em um dataframe chamado df. Alguns dos recursos de entrada e saída de interesse são:

- `nome`: Nome do país em questão.
- `continente` : 1=América do Norte, 2=América do Sul, 3=Europa, 4=África, 5=Ásia, 6=Oceania
- `barras`: Número de barras verticais na bandeira
- `listras` : Número de listras horizontais na bandeira
- `cores` : Número de cores diferentes na bandeira
- `vermelho` : 0 se vermelho ausente, 1 se vermelho presente na bandeira
...
- `cor_predominante` :  cor predominante na bandeira
- `círculos` : número de círculos na bandeira
- `cruzamentos` : Número de cruzamentos (na vertical)
- `cruzes` : número de cruzes diagonais
- `sol_estrelas` : Número de símbolos de sol ou estrela

**1.** Construiremos um classificador de árvore de decisão para prever de que continente vem uma determinada bandeira. Antes disso, queremos entender a distribuição das bandeiras por continente. Calcule a quantidade de países por  continente.

In [ ]:
df['continente1'] = 0
df['continente1'] = np.where(df['continente'] == 1, 1, df['continente1'])

df['continente2'] = 0
df['continente2'] = np.where(df['continente'] == 2, 1, df['continente2'])

df['continente3'] = 0
df['continente3'] = np.where(df['continente'] == 3, 1, df['continente3'])

df['continente4'] = 0
df['continente4'] = np.where(df['continente'] == 4, 1, df['continente4'])

df['continente5'] = 0
df['continente5'] = np.where(df['continente'] == 5, 1, df['continente5'])

df['continente6'] = 0
df['continente6'] = np.where(df['continente'] == 6, 1, df['continente6'])


In [ ]:
print("Quantidade de paises no continente 1:")
print(df['continente1'].value_counts()[1])

print("\n")

print("Quantidade de paises no continente 2:")
print(df['continente2'].value_counts()[1])

print("\n")

print("Quantidade de paises no continente 3:")
print(df['continente3'].value_counts()[1])

print("\n")

print("Quantidade de paises no continente 4:")
print(df['continente4'].value_counts()[1])

print("\n")

print("Quantidade de paises no continente 5:")
print(df['continente5'].value_counts()[1])

print("\n")

print("Quantidade de paises no continente 6:")
print(df['continente6'].value_counts()[1])

print("\n")

Quantidade de paises no continente 1:
31


Quantidade de paises no continente 2:
17


Quantidade de paises no continente 3:
35


Quantidade de paises no continente 4:
52


Quantidade de paises no continente 5:
39


Quantidade de paises no continente 6:
20




**2.** Em vez de olhar para todos os seis continentes, vamos nos concentrar em apenas dois, Europa e Oceania. Crie um novo dataframe apenas com bandeiras da Europa e Oceania.

In [ ]:
df_eurania = df[(df['continente'] == 3) | (df['continente'] == 6)]
df_eurania[['continente', 'nome']].groupby(['continente']).count()

NameError: ignored

**3.** Não usaremos todos os recursos disponíveis pois não são relevantes para classificação e/ou deixariam a árvore de decisão com muito profundidade. Crie um novo dataframe, chamado `x`, com somente as seguintes variáveis preditoras:

- `barras`
- `listras`
- `vermelho`
- `verde`
- `azul`
- `dourado`
- `branco`
- `preto`
- `laranja`
- `cor_predominante`
- `círculos`
- `cruzamentos`
- `cruzes`
- `quarters`
- `sol_estrelas`
- `triangle`
- `animate`

In [ ]:
columns = [
    'barras',
    'listras',
    'vermelho',
    'verde',
    'azul',
    'dourado',
    'branco',
    'preto',
    'laranja',
    'cor_predominante',
    'círculos',
    'cruzamentos',
    'cruzes',
    'quarters',
    'sol_estrelas',
    'triangle',
    'animate'
]
x = df_eurania[columns]

**4.** Construiremos um classificador para distinguir bandeiras para esses dois continentes – mas primeiro, imprima os tipos de dados das variáveis preditoras.

In [ ]:
x.info()

**5.** Observe que todas as variáveis de previsão são numéricas, exceto `cor_predominante`. Transforme o conjunto de dados de variáveis preditoras em variáveis fictícias usando a função do pandas `get_dummies`.

In [ ]:
df_cor = pd.get_dummies(x['cor_predominante'].unique(), drop_first=True)

In [ ]:
x = pd.merge(x, df_cor, left_index=True, right_index=True)

In [ ]:
x.drop(['cor_predominante'], axis=1, inplace=True)

**6.** Armazene os dados do atributo de saída na variável `y` 

In [ ]:
y = df_eurania['continente']

**7.** Divida os dados em conjuntos de treino e teste.

In [ ]:
#Dividindo o conjunto entre treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

### Ajustar classificadores de árvore de decisão por profundidade

**8.** Ajuste o modelo de árvore de decisão testando o desempenho em uma faixa de valores de profundidade máxima. Ajuste um classificador de árvore de decisão para valores de `max_depth` de 1 a 20. Salve a pontuação de acurácia para cada profundidade na lista `profundidade`

In [ ]:
max_depth_settings = range(1, 20)
training_accuracy = []
test_accuracy = []

for n in max_depth_settings:
  model = DecisionTreeClassifier(max_depth=n, random_state=0)
  model.fit(x_train, y_train)
  training_accuracy.append(model.score(x_train, y_train))
  test_accuracy.append(model.score(x_test, y_test))

**9.** Plote a acurácia dos modelos de árvore de decisão versus o `max_depth`.

In [ ]:
plt.plot(max_depth_settings, training_accuracy, label='Acuracia Training Set')
plt.plot(max_depth_settings, test_accuracy, label='Acuracia Test Set')
plt.ylabel('Acuracia')
plt.xlabel('max_depth')
plt.legend()

**10.** Imprima a acurácia e profundidade do modelo com a maior precisão.

In [ ]:
melhor_max_depth = 6
model = DecisionTreeClassifier(max_depth=melhor_max_depth, random_state=0)
model.fit(x_train, y_train)

print("Melhor max depth: {}".format(melhor_max_depth))
print("Acuracia train data: {}".format(model.score(x_train, y_train)))
print("Acuracia test data: {}".format(model.score(x_test, y_test)))


**11.** Reajuste o modelo de árvore de decisão usando o `max_depth` que obteve a maior acurácia; Mostre a árvore de decisão resultante.

In [ ]:
from sklearn.tree import plot_tree

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=500)
tree.plot_tree(model, feature_names = x.columns, class_names=['europa', 'oceania'], filled = True);

**12.** Existem algumas maneiras diferentes de estender este projeto. Tente classificar outra coisa! Por exemplo, em vez de prever o `continente`, poderia prever algo como o `idioma`.

Ajuste mais parâmetros do modelo. Você pode encontrar uma descrição de todos os parâmetros que podem ser ajustados na [documentação do Classificador de Árvore de Decisão](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier). Por exemplo, veja o que acontece se você ajustar o `max_leaf_nodes`. Pense se você estaria superajustando ou subajustando os dados com base em quantos nós folha você permite.